In [1]:
import csv
import numpy as np
import pandas as pd
# import geopandas as gpd
from IPython.display import Image
# from shapely.geometry import Point, Polygon
from math import factorial
import datetime
import time
import scipy
import os, os.path
import itertools
from statsmodels.sandbox.regression.predstd import wls_prediction_std
from sklearn.linear_model import LinearRegression
from patsy import cr

# from pprint import pprint
import matplotlib.pyplot as plt
import seaborn as sb


import sys
start_time = time.time()

In [2]:
sys.path.append('/Users/hn/Documents/00_GitHub/Ag/remote_sensing/python/')
import remote_sensing_core as rc

# sys.path.append('/home/hnoorazar/remote_sensing_codes/')
# import remote_sensing_core as rcp

In [3]:
param_dir = "/Users/hn/Documents/00_GitHub/Ag/remote_sensing/parameters/"

double_crop_potens = pd.read_csv(param_dir + "double_crop_potential_plants.csv")

#### Read ShapeFile Data

In [4]:
shapeFile_Data_dir = "/Users/hn/Documents/01_research_data/" + \
                      "remote_sensing/01_Data_part_not_filtered/"

WSDA_DataTable_2018 = pd.read_csv(shapeFile_Data_dir + "WSDA_DataTable_2018.csv")

WSDA_DataTable_2018["DataSrc"] = WSDA_DataTable_2018["DataSrc"].str.lower()
WSDA_DataTable_2018["CropTyp"] = WSDA_DataTable_2018["CropTyp"].str.lower()

In [5]:
indeks = "EVI"

peak_tables_dir = "/Users/hn/Documents/01_research_data/" + \
                  "remote_sensing/01_NDVI_TS/04_Irrigated_eastern_Cloud70/" + \
                  "Grant_2018_irrigated/savitzky_" + indeks + "/Grant_Irrigated_2018_no_plot/"

####
####  parameters
####
deltas = [0.1, 0.2, 0.3, 0.4]
windows_degrees = [(3, 1), (3, 2),
                   (5, 1), (5, 2), (5, 3),
                   (7, 1), (7, 2), (7, 3), 
                   (9, 1), (9, 2), (9, 3)]

windows_degrees = [[3, 1], [3, 2],
                   [5, 1], [5, 2], [5, 3],
                   [7, 1], [7, 2], [7, 3], 
                   [9, 1], [9, 2], [9, 3]]



delta_windows_degrees = [[i, j] for i in deltas for j in windows_degrees]

output_columns = ['parameters', 'actual_2_pred_2', 'actual_2_pred_Not2',
                  'actual_Not2_pred_2', 'actual_Not2_pred_Not2',]

In [6]:
WSDA_DataTable_2018.DataSrc.unique()

array(['nass', 'wsda', 'conservation district', 'producer', 'king county',
       'wslcb', 'snohomish county', 'doh'], dtype=object)

In [7]:
NASS_out = False
non_Irr_out = True
perennials_out = False

output = pd.DataFrame(data=None, 
                      index = np.arange(len(delta_windows_degrees)), # delta_windows_degrees, # 
                      columns = output_columns)
output['parameters'] = delta_windows_degrees
curr_SF = WSDA_DataTable_2018.copy()

if perennials_out == True:
    curr_SF = curr_SF[curr_SF.CropTyp.isin(double_crop_potens['Crop_Type'])]
    Pere_name = "PereOut_"
else:
    Pere_name = "PereIn_"

if NASS_out == True:
    curr_SF = rc.filter_out_NASS(curr_SF)
    NASS_name = "NASSOut_"
else:
    NASS_name = "NASSin_"

In [ ]:
if non_Irr_out == True:
    curr_SF = rc.filter_out_nonIrrigated(curr_SF)
    non_Irr_name = "JustIrr"
else:
    non_Irr_name = "BothIrr"



In [8]:
curr_SF.shape

(212985, 16)

In [9]:
curr_SF = rc.filter_out_nonIrrigated(curr_SF)

In [10]:
curr_SF.shape

(84717, 16)

In [ ]:
print ("NASS_out: " + str(NASS_out) + ", non_Irr_out: " + str(non_Irr_out) + \
       ", perennials_out: " + str(perennials_out))

location = 0
params = output['parameters'][0]



In [ ]:
delt = params[0]
win_deg = params[1]
window = win_deg[0]
degree = win_deg[1]
doubl_pk_file = peak_tables_dir + "delta" + str(delt) + "_Sav_win" + str(window) + \
                "_Order" + str(degree) + "/all_poly_and_maxs_savitzky.csv"

doubl_peak_table = pd.read_csv( doubl_pk_file, low_memory=False)

doubl_peak_table["CropTyp"] = doubl_peak_table["CropTyp"].str.lower()

############################################################
last_row_id = doubl_peak_table.shape[0] - 1
doubl_peak_table = doubl_peak_table.drop([last_row_id], axis=0)

if perennials_out == True:
    doubl_peak_table = doubl_peak_table[\
                                    doubl_peak_table.CropTyp.isin(double_crop_potens['Crop_Type'])]
if NASS_out == True:
    doubl_peak_table = rc.filter_out_NASS(doubl_peak_table)


if non_Irr_out == True:
    doubl_peak_table = rc.filter_out_nonIrrigated(doubl_peak_table)

In [ ]:
doubl_peak_table.drop(['max_Doy', 'max_value'], axis=1, inplace=True)
doubl_peak_table.drop_duplicates(inplace=True)

In [ ]:
#### 
actual_double_cropped = rc.filter_double_by_Notes(curr_SF)
actual_Notdouble_cropped = rc.filter_Notdouble_by_Notes(curr_SF)

In [ ]:
actual_double_cropped.head(2)

In [ ]:
actual_Notdouble_cropped.head(2)

In [ ]:
predicted_double_peaks = doubl_peak_table[doubl_peak_table.max_count >= 2].copy()
predicted_Notdouble_peaks = doubl_peak_table[doubl_peak_table.max_count < 2].copy()

In [ ]:
predicted_Notdouble_peaks.head(2)

In [ ]:
actual_2_pred_2 = sum(actual_double_cropped['ID'].isin(predicted_double_peaks['ID']))
actual_Not2_pred_2 = sum(actual_Notdouble_cropped['ID'].isin(predicted_double_peaks['ID']))


In [ ]:
actual_2_pred_Not2 = len(actual_double_cropped['ID'].unique()) - actual_2_pred_2
actual_Not2_pred_Not2 = len(actual_Notdouble_cropped['ID'].unique()) - actual_Not2_pred_2

In [ ]:
print(actual_2_pred_2)
print(actual_Not2_pred_2)
print(actual_2_pred_Not2)
print(actual_Not2_pred_Not2)

In [ ]:
print(NASS_out)
print(non_Irr_out)
print(perennials_out)

In [ ]:
actual_Notdouble_cropped.head(2)

In [ ]:
len(actual_Notdouble_cropped.ID.unique()) == len(actual_Notdouble_cropped.ID)

In [ ]:
fillin_col = ["actual_2_pred_2", "actual_2_pred_Not2", "actual_Not2_pred_2", "actual_Not2_pred_Not2"]
fillin_vals = [actual_2_pred_2, actual_2_pred_Not2, actual_Not2_pred_2, actual_Not2_pred_Not2]
output.loc[location, fillin_col] = fillin_vals

In [ ]:
###########
output['parameters'] = output['parameters'].astype("str")
write_path = "/Users/hn/Documents/fuck_1/"
write_path = "/Users/hn/Documents/fuck/"
os.makedirs(write_path, exist_ok=True)

filename = write_path + "saviztky_" + indeks + "_" + Pere_name + \
           NASS_name + non_Irr_name + "_confusion.csv"
output.to_csv(filename, index = False)
del(output)

In [9]:
jumps = "no"
SF_year = 2017
output_base = "a/"
output_Irr = "Irr"
indeks = "EVI"
sos_thresh = 0.4
eos_thresh = 0.4
regular_data_dir = "100"
given_county = "Grant"

In [10]:
if jumps == "yes":
    regular_output_dir = output_base + "/2Yrs_plots_70Cloud_Regular_wJumps/" + \
                         str(SF_year) + "_regular_" + output_Irr + "_" + indeks + \
                         "_SOS" + str(int(sos_thresh*10))+ "_EOS" + str(int(eos_thresh*10)) + "/"
else:
    regular_data_dir = regular_data_dir + "/noJump_Regularized/"
    regular_output_dir = output_base + "/2Yrs_plots_70Cloud_Regular_noJumps/" + \
                         str(SF_year) + "_regular_" + output_Irr + "_" + indeks + \
                         "_SOS" + str(int(sos_thresh*10)) + "_EOS" + str(int(eos_thresh*10)) + "/"

    f_name = "01_Regular_filledGap_" + given_county + "_SF_" + str(SF_year) + "_" + indeks + ".csv"

In [1]:
import pandas as pd

A = "/Users/hn/Documents/01_research_data/remote_sensing/" + \
    "01_NDVI_TS/70_Cloud/00_Eastern_WA_withYear/2Years/" + \
    "04_Regular_plt_tbl_SOSEOS/confusions_70Cloud_SEOS0.5/" + \
    "EVI_PereOut_NASSin_JustIrr_dblNotFiltered_confusion_Acr_morethan2seasons_regular/" + \
    "allCounties_separate_2016_confusion.csv"

A = pd.read_csv(A)

In [3]:
eastern_confusion = A.groupby(['parameters']).sum()

In [4]:
eastern_confusion

,actual_2_pred_2,actual_2_pred_Not2,actual_Not2_pred_2,actual_Not2_pred_Not2
parameters,,,,
"[5, 1]",8714.76,17439.52,105751.22,705202.20
"[5, 3]",9528.23,16626.05,141943.79,669009.60
"[7, 3]",10144.48,16009.80,142171.67,668781.73
"[9, 3]",10345.40,15808.89,134406.76,676546.64


In [15]:
columnss = A.columns[0:5]

In [16]:
eastern_confusion = eastern_confusion[columnss]

In [17]:
eastern_confusion

,parameters,actual_2_pred_2,actual_2_pred_Not2,actual_Not2_pred_2,actual_Not2_pred_Not2
parameters,,,,,
"[5, 1]","[5, 1]",8714.76,17439.52,105751.22,705202.20
"[5, 3]","[5, 3]",9528.23,16626.05,141943.79,669009.60
"[7, 3]","[7, 3]",10144.48,16009.80,142171.67,668781.73
"[9, 3]","[9, 3]",10345.40,15808.89,134406.76,676546.64
